In [ ]:
!pip3 install gensim

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
import nltk
import pandas as pd
import re
import numpy as np

In [ ]:
bot_users = pd.read_csv("./potential_social_botnets.csv", delimiter=',', encoding='ISO-8859-2')
users = pd.read_csv("./anonymized_disqus_user_data.csv", delimiter=',', encoding='ISO-8859-2')
users.drop_duplicates(subset='username', keep='first', inplace=True, ignore_index=True)
bot_comments = pd.read_csv("./botnet_comments.csv", delimiter=',', encoding='ISO-8859-2')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
eng_stopwords = nltk.corpus.stopwords.words("english") 
eng_stopwords.extend([u'lol', u'haha'])

In [ ]:
bot_users['creation_day'] = bot_users.apply(lambda x: x['creation_date'][:10], axis=1)

In [ ]:
import time
import datetime
from dateutil import parser

In [ ]:
def date_to_utc_seconds(s):
    s = str(s) + ':00'
    date = parser.parse(s)
    return int(time.mktime(date.timetuple()))

In [ ]:
bot_users['creation_date_utc'] = bot_users.apply (lambda row: date_to_utc_seconds(row['creation_date']), axis=1)

In [ ]:
def get_cluster_id(row, user_df):
    x = user_df.loc[user_df['username'] == row['username']]
    if x.empty:
        return None
    return x.iloc[0]['cluster_id']

In [ ]:
user_features = {} # username: [wc1, wc2..., date_created_utc]
seen_words = set() # set of all nouns

In [ ]:
is_noun = lambda pos: pos[:2] == 'NN'
def clean_comment(row):
    s = str(row['message'])
    s = re.sub("<[^>]*>", " ", s.strip().lower())
    s = ' '.join([word for word in s.split() if word not in eng_stopwords])
    s = remove_stopwords(s)
    s_tokens = nltk.word_tokenize(s)
    nouns = [word for (word, pos) in nltk.pos_tag(s_tokens) if is_noun(pos)]
    for word in nouns:
          seen_words.add(word)      
    return ' '.join(nouns)

In [ ]:
bot_comments_sample = bot_comments.sample(50000)

In [ ]:
bot_comments_sample['noun_message'] = bot_comments_sample.apply (lambda row: clean_comment(row), axis=1)

In [ ]:
seen_words_dict = {}
seen_words_list = list(seen_words)
for idx, word in enumerate(seen_words_list):
    seen_words_dict[word] = idx

In [ ]:
def make_features(row):
    if row['username'] in user_features:
        features = user_features[row['username']]
    else:
        features = [0]*len(seen_words_list)
    
    nouns = row['noun_message'].split()
    for word in nouns:
        features[seen_words_dict[word]] += 1
    user_features[row['username']] = features
    

In [ ]:
dont_print = bot_comments_sample.apply (lambda row: make_features(row), axis=1)

In [ ]:
bot_users['creation_date_utc_delta'] = bot_users.apply (lambda row: row['creation_date_utc'] - 1199880694, axis=1)

In [ ]:
def add_features(row):
    features = None
    if row['username'] in user_features:
        features = user_features[row['username']]
        features.append(row['creation_date_utc_delta'])
    return features


In [ ]:
bot_users['features'] = bot_users.apply (lambda row: add_features(row), axis=1)

In [ ]:
bot_users_with_features = bot_users.dropna(subset=['features'])

In [ ]:
bot_users_with_features.head(1)

In [ ]:
bot_users_with_features.head(1)

In [ ]:
import sklearn.cluster as cluster

In [ ]:
kmeans = cluster.KMeans(n_clusters=50, init='k-means++')

In [ ]:
(pd.DataFrame(bot_users_with_features['features'].values.tolist()).iloc[:, :-1])

In [ ]:
kmeansx = kmeans.fit(pd.DataFrame(bot_users_with_features['features'].values.tolist()).iloc[:, :-1])

In [ ]:
bot_users_with_features['cluster_id'] = kmeansx.labels_

In [ ]:
bot_users_with_features.head(10)

In [ ]:
bot_comments_sample['cluster'] = bot_comments_sample.apply(lambda row: get_cluster_id(row, bot_users_with_features), axis=1)

In [ ]:
bot_comments_sample.head(20)

In [ ]:
bot_comments_sample = bot_comments_sample.dropna(subset=['cluster'])

In [ ]:
bot_comments_sample.head(10)

In [ ]:
x = bot_comments_sample.groupby('cluster')

In [ ]:
x.apply(display)